# Import and load files

In [1]:
# model contains trained parameters by facenet... Input dimension (160,160,3,) Ouput dimension (128,)
# trained_model contains parameters after training images through facenet and then through soft_train... Input dimension (128,) 
# Output dimension (32, )
import cv2
from keras.models import load_model
import numpy as np
model = load_model('facenet_keras.h5')
trained_model = load_model('soft_train.h5')

Using TensorFlow backend.
C:\Users\HP\.conda\envs\gputest\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# Highlight faces in Image

In [2]:
# Function to detect faces and highlight faces with rectangular boxes
def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    
    detections=net.forward()
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn,faceBoxes

# Load face model

In [3]:
# Load face model to detect faces and pass to  highlightFace function
faceProto="opencv_face_detector.pbtxt"
faceModel="opencv_face_detector_uint8.pb"
faceNet=cv2.dnn.readNet(faceModel,faceProto)

# Extract face

In [4]:
# Extract face and resize image into (160,160)
def get_face(faceBoxes,frame):
    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):
                    min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding)
                    :min(faceBox[2]+padding, frame.shape[1]-1)]
#     resizing the pixels to the model size

    image = Image.fromarray(face.astype('uint8'))
    image = image.resize((160,160))
    face_array = np.array(image)
    return face_array

# Get vector inputs from 3-D Images

In [5]:
# Get image embedding through facenet and through soft_train model (Vector Prediction of extracted faces)
def get_embedding(model, trained_model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    face_pixels = face_pixels/255
    # standarized pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = np.expand_dims(face_pixels, axis = 0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    X = np.expand_dims(yhat[0], axis = 0)
    pred = trained_model.predict(X)
    return pred[0]

# Create a Database

In [6]:
# from PIL import Image
# import numpy as np
# %matplotlib inline
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# padding = 20
# # Create a database with a name and corresponding embedded images for face recognition
# database = {}
# name = ['subin', 'selena', 'ben', 'jerry', 'taylor', 'mindy', 'krijan']
# for i in range(7):
#     image = Image.open('database/{}.jpg'.format(i))
#     image = np.array(image.convert('RGB'))
#     resultImg,faceBoxes=highlightFace(faceNet,image)
#     face = get_face(faceBoxes,image)
#     emb = get_embedding(model,trained_model, face)
#     database[name[i]] = emb
    
# # Plot images
# plt.subplot(1,2,1)
# plt.imshow(resultImg)
# plt.subplot(1,2,2)
# plt.imshow(face)
# plt.show()

In [8]:
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
padding = 20

img_num = 20
# zx = random.uniform(0,0.05)
# zy = random.uniform(0,0.05)
datagen = ImageDataGenerator()
# Create a database with a name and corresponding embedded images for face recognition
database = {}
name = ['subin', 'selena', 'ben', 'jerry', 'taylor', 'mindy', 'krijan']
for i in range(7):
    image = Image.open('database/{}.jpg'.format(i))
    image = np.array(image.convert('RGB'))
    

    emb = 0
    resultImg,faceBoxes=highlightFace(faceNet,image)
    face = get_face(faceBoxes,image)
    emb += get_embedding(model,trained_model, face)
    
    #create multiple image of a same image with different brightness and rotation
    for j in range(img_num):
        theta = random.randint(0,45)
        brightness = random.uniform(0.3,1)
        image = Image.open('database/{}.jpg'.format(i))
        image = np.array(image.convert('RGB'))
        image = datagen.apply_transform(x=image, transform_parameters={'theta':theta, 'brightness':brightness, 
                                                                   })

        resultImg,faceBoxes=highlightFace(faceNet,image)
        face = get_face(faceBoxes,image)
        
#         plt.imshow(face/255)
#         plt.show()

        emb += get_embedding(model,trained_model, face)
        
    emb = emb / (img_num+1)
    
    database[name[i]] = emb
    
# Plot images
# plt.subplot(1,2,1)
# plt.imshow(resultImg)
# plt.subplot(1,2,2)
# plt.imshow(face)
# plt.show()

# Verify Images

In [9]:
# Verify current image with the images in the database...
# Image is same if the norm L1 distance between them is less than 0.6...

def verify(image_path, identity, database, model,trained_model):
#     image = Image.open(image_path)
#     image = np.array(image.convert('RGB'))
    image = image_path.copy()
    
    resultImg,faceBoxes=highlightFace(faceNet,image)
    face = get_face(faceBoxes,image)
    encoding = get_embedding(model,trained_model,face)
    
    dist = np.linalg.norm(encoding - database[identity], ord= 1)

    if dist < 0.55:
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    return dist, door_open

In [10]:
# # Checking one of the images
# verify('2.jpg','ben',database,model,trained_model)

# Capture images from Camera and Verify with Database

In [25]:
# Capture images from video and verify with database....
video=cv2.VideoCapture(0)
while cv2.waitKey(1)<0:
    hasFrame,frame=video.read()
    if not hasFrame:
        cv2.waitKey()
        break

    resultImg,faceBoxes=highlightFace(faceNet,frame)

    if not faceBoxes:
        print("No face detected")

    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):
                   min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding)
                   :min(faceBox[2]+padding, frame.shape[1]-1)]

        dist, door = verify(frame,'ben',database,model,trained_model)

    cv2.putText(resultImg, f'{dist}, {door}', (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2, cv2.LINE_AA)
    cv2.imshow("Detecting face", resultImg)
    
    if door:
        print("welcome")
        break
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, please go away
It's not ben, 

In [12]:
# from keras.preprocessing.image import ImageDataGenerator
# from PIL import Image
# import numpy as np
# import random
# theta = random.randint(0,90)
# brightness = random.uniform(0,1)
# zx = random.uniform(0,1)
# zy = random.uniform(0,1)
# image_path = "2.jpg"
# image = Image.open(image_path)
# image = np.array(image.convert('RGB'))
# datagen = ImageDataGenerator()
# X = 0
# for i in range(4):
#     X += datagen.apply_transform(x=image, transform_parameters={'theta':theta, 'brightness':brightness, 
#                                                                     'zx':zx, 'zy':zy})
# print(image)